In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!unzip "/content/drive/My Drive/Content-Summary_CleanData.zip" -d "/content/extracted_files"

Archive:  /content/drive/My Drive/Content-Summary_CleanData.zip
replace /content/extracted_files/Content-Summary_CleanData.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [1]:
import pandas as pd

df = pd.read_csv('/content/extracted_files/Content-Summary_CleanData.csv')
df.head()

,Content,Summary
0,New York police are concerned drones could bec...,Police have investigated criminals who have ri...
1,By . Ryan Lipman . Perhaps Australian porn sta...,Porn star Angela White secretly filmed sex act...
2,"This was, Sergio Garcia conceded, much like be...",American draws inspiration from fellow country...
3,An Ebola outbreak that began in Guinea four mo...,World Health Organisation: 635 infections and ...
4,By . Associated Press and Daily Mail Reporter ...,A sinkhole opened up at 5:15am this morning in...


In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['Summary'].value_counts()
    for x_label, grp in _df_2.groupby('Content')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('Content')
_ = plt.ylabel('Summary')

In [12]:
#  install unsloth , xformers (flash attention) and all other packages !
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# we have to check which torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"

!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-duvfmzwa/unsloth_478a1fb27ac54cca813f09b722b71bc5
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-duvfmzwa/unsloth_478a1fb27ac54cca813f09b722b71bc5
  Resolved https://github.com/unslothai/unsloth.git to commit 8558bc92b06f9128499484ef737fa71b966ffc23
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.1/112.1 kB 8.0 MB/s eta 0:00:00
   ━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 64.1 MB/s eta 0:00:00
ERROR: Operation cancelled by user


In [5]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model,tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.11.10: Fast Llama patching. Transformers:4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

Unsloth 2024.11.10 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [7]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Your task is to understand the provided text in English and generate a summary and a title for it in english. The result should be formatted in JSON. Follow these steps:

1- Comprehension:
Understand the provided English text.
2- Summary Generation:
Create a brief and concise summary of the text in English.
3- Title Generation:
Generate a relevant title in English that accurately reflects the content and theme of the text.
4-Output Formatting:
Format the summary and title into a JSON object with the keys 'summary' and 'title'.

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    inputs       = examples["Content"]
    outputs      = examples["Summary"]
    texts = []
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [8]:
# reducing data set from 600k to 1000
df_subset = df.iloc[:1000]
df_subset.to_csv("summary1000.csv", index=False)

In [9]:
df_1000 = pd.read_csv('/content/summary1000.csv')

In [10]:
from sklearn.model_selection import train_test_split
from datasets import load_dataset
# Split your DataFrame into training and testing sets (80% train, 20% test)
train_df, test_df = train_test_split(df_1000, test_size=0.2)

# Save the splits to separate CSV files
train_df.to_csv('train_dataset.csv', index=False)
test_df.to_csv('test_dataset.csv', index=False)

# Load the datasets
train_dataset = load_dataset("csv", data_files="train_dataset.csv")
test_dataset = load_dataset("csv", data_files="test_dataset.csv")

print(train_dataset['train'])
print(test_dataset['train'])

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['Content', 'Summary'],
    num_rows: 800
})
Dataset({
    features: ['Content', 'Summary'],
    num_rows: 200
})


In [11]:
train_dataset['train']

Dataset({
    features: ['Content', 'Summary'],
    num_rows: 800
})

In [12]:
#convert the dataset to dataframe
train_df = train_dataset['train'].to_pandas(train_dataset['train'])

In [13]:
train_df.shape

(800, 2)

In [14]:
import pandas as pd
from datasets import Dataset

# convert the dataframe to dataset
dataset = Dataset.from_pandas(train_df)

In [15]:
dataset

Dataset({
    features: ['Content', 'Summary'],
    num_rows: 800
})

In [16]:
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

# Training Model

In [17]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,         # reducing steps
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/800 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [19]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")


GPU = Tesla T4. Max memory = 14.748 GB.
6.004 GB of memory reserved.


In [20]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 800 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1,2.029800
2,2.106400
3,2.141600
4,2.062000
5,2.049200
6,1.902200
7,2.023600
8,1.962600
9,1.896200
10,1.701000


saving model

In [21]:
trainer.save_model('/content/drive/MyDrive/awesome_news_summary_model')

In [22]:
trainer.save_model('/content/awesome_news_summary_model')


# Show final memory and time stats

In [23]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1938.1435 seconds used for training.
32.3 minutes used for training.
Peak reserved memory = 8.096 GB.
Peak reserved memory for training = 2.092 GB.
Peak reserved memory % of max memory = 54.896 %.
Peak reserved memory for training % of max memory = 14.185 %.


# Inference

In [32]:
# article  = df.sample(1).iloc[0]['Content']

article ="""
The decision to buy a flagship or luxury product is often driven more by brand value than the product’s true value, a phenomenon known as brand bias. As a result, we may choose a less superior option even when better alternatives are available at a better value. In the smartphone industry, iQOO stands out as a brand that focuses on delivering exceptional value that some might overlook. Over the years, iQOO has been consistently releasing flagship Android smartphones that outperform the competition while being much more affordable, and their latest model, the iQOO 13, continues this trend. It offers incredible value without compromising on the key features essential in a flagship device today. You have exhausted your monthly limit of free stories. Register to read more stories Sign up Already have an account? Sign in
"""

In [33]:

# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        article, # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

["<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nYour task is to understand the provided text in English and generate a summary and a title for it in english. The result should be formatted in JSON. Follow these steps:\n\n1- Comprehension:\nUnderstand the provided English text.\n2- Summary Generation:\nCreate a brief and concise summary of the text in English.\n3- Title Generation:\nGenerate a relevant title in English that accurately reflects the content and theme of the text.\n4-Output Formatting:\nFormat the summary and title into a JSON object with the keys'summary' and 'title'.\n\n### Input:\n\nThe decision to buy a flagship or luxury product is often driven more by brand value than the product’s true value, a phenomenon known as brand bias. As a result, we may choose a less superior option even when better alternatives are available 

In [36]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        article, # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 120)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Your task is to understand the provided text in English and generate a summary and a title for it in english. The result should be formatted in JSON. Follow these steps:

1- Comprehension:
Understand the provided English text.
2- Summary Generation:
Create a brief and concise summary of the text in English.
3- Title Generation:
Generate a relevant title in English that accurately reflects the content and theme of the text.
4-Output Formatting:
Format the summary and title into a JSON object with the keys'summary' and 'title'.

### Input:

The decision to buy a flagship or luxury product is often driven more by brand value than the product’s true value, a phenomenon known as brand bias. As a result, we may choose a less superior option even when better alternatives are available at a better value.

# Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's push_to_hub for an online save or save_pretrained for a local save.


In [ ]:

model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
model.push_to_hub("repo\model_name", token = "...") # Online saving
tokenizer.push_to_hub("repo\model_name", token = "...") # Online saving

In [7]:

model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
model.push_to_hub("repo\model_name", token = "...") # Online saving
tokenizer.push_to_hub("repo\model_name", token = "...") # Online saving

NameError: name 'trainer_stats' is not defined

In [3]:
from huggingface_hub import notebook_login
notebook_login()

NameError: name 'model' is not defined

In [ ]:
model.save_pretrained("awesome_news_summary_model") # Local saving
tokenizer.save_pretrained("awesome_news_summary_model")
model.push_to_hub("repo\model_name", token = "...") # Online saving
tokenizer.push_to_hub("repo\model_name", token = "...") # Online saving

# loading model adapters , using inference we just saved

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "AhmedBou/Llama-3-EngNewsSummary", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

inputs = tokenizer(
[
    alpaca_prompt.format(
        "", # input news article
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)